In [1]:
import pandas as pd

# metadata.xlsx dosyasını oku
df = pd.read_excel("metadata.xlsx")

# İlk satırlara bakalım
display(df.head())

# Boyut ve kolon isimleri
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())


,file_name,text,speaker_id,is_common,sentence_index
0,Speaker0_00.m4a,Bugün hava oldukça sakin.,0,1,0
1,Speaker0_01.m4a,Toplantı saatini üç buçuğa erteledim.,0,1,1
2,Speaker0_02.m4a,Yoğun trafik nedeniyle otobüs yarım saatten fa...,0,1,2
3,Speaker0_03.m4a,Bu dosyayı ne zaman teslim etmemiz gerekiyor?,0,1,3
4,Speaker0_04.m4a,"E-posta adresimi yanlış yazmış olabilirim, tek...",0,1,4


Shape: (100, 5)
Columns: ['file_name', 'text', 'speaker_id', 'is_common', 'sentence_index']


In [2]:
df.to_csv("metadata.csv", index=False, encoding="utf-8")
print("metadata.csv oluşturuldu")


metadata.csv oluşturuldu


In [3]:
import pandas as pd
import os

df = pd.read_csv("metadata.csv")

AUDIO_DIR = "raw_audio-20251213T205657Z-3"

df["audio_path"] = df["file_name"].apply(
    lambda x: os.path.join(AUDIO_DIR, x)
)

# Kontrol
display(df[["audio_path", "text"]].head())


,audio_path,text
0,raw_audio-20251213T205657Z-3/Speaker0_00.m4a,Bugün hava oldukça sakin.
1,raw_audio-20251213T205657Z-3/Speaker0_01.m4a,Toplantı saatini üç buçuğa erteledim.
2,raw_audio-20251213T205657Z-3/Speaker0_02.m4a,Yoğun trafik nedeniyle otobüs yarım saatten fa...
3,raw_audio-20251213T205657Z-3/Speaker0_03.m4a,Bu dosyayı ne zaman teslim etmemiz gerekiyor?
4,raw_audio-20251213T205657Z-3/Speaker0_04.m4a,"E-posta adresimi yanlış yazmış olabilirim, tek..."


In [4]:
missing = df[~df["audio_path"].apply(os.path.exists)]
print("Eksik dosya sayısı:", len(missing))

missing.head()


Eksik dosya sayısı: 100


,file_name,text,speaker_id,is_common,sentence_index,audio_path
0,Speaker0_00.m4a,Bugün hava oldukça sakin.,0,1,0,raw_audio-20251213T205657Z-3/Speaker0_00.m4a
1,Speaker0_01.m4a,Toplantı saatini üç buçuğa erteledim.,0,1,1,raw_audio-20251213T205657Z-3/Speaker0_01.m4a
2,Speaker0_02.m4a,Yoğun trafik nedeniyle otobüs yarım saatten fa...,0,1,2,raw_audio-20251213T205657Z-3/Speaker0_02.m4a
3,Speaker0_03.m4a,Bu dosyayı ne zaman teslim etmemiz gerekiyor?,0,1,3,raw_audio-20251213T205657Z-3/Speaker0_03.m4a
4,Speaker0_04.m4a,"E-posta adresimi yanlış yazmış olabilirim, tek...",0,1,4,raw_audio-20251213T205657Z-3/Speaker0_04.m4a


In [6]:
import os
os.listdir(".")


['.config',
 '.ipynb_checkpoints',
 'raw_audio-20251213T205657Z-3-001.zip',
 'metadata.csv',
 'metadata.xlsx']

In [7]:
!unzip -q "raw_audio-20251213T205657Z-3-001.zip" -d raw_audio


In [8]:
import os
os.listdir(".")
print("raw_audio içi:", os.listdir("raw_audio")[:20])


raw_audio içi: ['raw_audio']


In [9]:
import glob

m4a_files = glob.glob("raw_audio/**/*.m4a", recursive=True)
print("Bulunan m4a sayısı:", len(m4a_files))
print("Örnek:", m4a_files[:5])


Bulunan m4a sayısı: 100
Örnek: ['raw_audio/raw_audio/Speaker1_12.m4a', 'raw_audio/raw_audio/Speaker1_14.m4a', 'raw_audio/raw_audio/Speaker1_01.m4a', 'raw_audio/raw_audio/Speaker4_19.m4a', 'raw_audio/raw_audio/Speaker3_17.m4a']


In [10]:
import pandas as pd
import os
import glob

df = pd.read_csv("metadata.csv")

# bütün m4a dosyalarını bul
m4a_files = glob.glob("raw_audio/**/*.m4a", recursive=True)

# file_name -> tam path map'i (dosya adına göre)
name2path = {os.path.basename(p): p for p in m4a_files}

# metadata'ya path ekle
df["audio_path"] = df["file_name"].map(name2path)

missing = df[df["audio_path"].isna()]
print("Eşleşmeyen dosya sayısı:", len(missing))
display(missing.head())

display(df[["audio_path", "text"]].head())


Eşleşmeyen dosya sayısı: 0


,file_name,text,speaker_id,is_common,sentence_index,audio_path


,audio_path,text
0,raw_audio/raw_audio/Speaker0_00.m4a,Bugün hava oldukça sakin.
1,raw_audio/raw_audio/Speaker0_01.m4a,Toplantı saatini üç buçuğa erteledim.
2,raw_audio/raw_audio/Speaker0_02.m4a,Yoğun trafik nedeniyle otobüs yarım saatten fa...
3,raw_audio/raw_audio/Speaker0_03.m4a,Bu dosyayı ne zaman teslim etmemiz gerekiyor?
4,raw_audio/raw_audio/Speaker0_04.m4a,"E-posta adresimi yanlış yazmış olabilirim, tek..."


In [47]:
asr_df = df[["audio_path", "text"]].copy()
asr_df["text"] = asr_df["text"].astype(str).str.strip()

asr_df.to_csv("asr_metadata.csv", index=False, encoding="utf-8")
print(asr_df.shape)
asr_df.head()


(100, 2)


,audio_path,text
0,raw_audio/raw_audio/Speaker0_00.m4a,Bugün hava oldukça sakin.
1,raw_audio/raw_audio/Speaker0_01.m4a,Toplantı saatini üç buçuğa erteledim.
2,raw_audio/raw_audio/Speaker0_02.m4a,Yoğun trafik nedeniyle otobüs yarım saatten fa...
3,raw_audio/raw_audio/Speaker0_03.m4a,Bu dosyayı ne zaman teslim etmemiz gerekiyor?
4,raw_audio/raw_audio/Speaker0_04.m4a,"E-posta adresimi yanlış yazmış olabilirim, tek..."


In [12]:
!apt-get -qq update
!apt-get -qq install -y ffmpeg


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [49]:
import os
import pandas as pd

os.makedirs("wav16k", exist_ok=True)

asr_df = pd.read_csv("asr_metadata.csv")

def m4a_to_wav16k(m4a_path):
    base = os.path.splitext(os.path.basename(m4a_path))[0]
    wav_path = f"wav16k/{base}.wav"
    return wav_path

asr_df["wav_path"] = asr_df["audio_path"].apply(m4a_to_wav16k)

# dönüşüm
for m4a_path, wav_path in zip(asr_df["audio_path"], asr_df["wav_path"]):
    if not os.path.exists(wav_path):
        !ffmpeg -y -loglevel error -i "$m4a_path" -ac 1 -ar 16000 "$wav_path"

print("Bitti. Örnek wav:", asr_df["wav_path"].iloc[0])


Bitti. Örnek wav: wav16k/Speaker0_00.wav


In [48]:
import os, glob
print("wav sayısı:", len(glob.glob("wav16k/*.wav")))


wav sayısı: 100


In [15]:
print(asr_df.columns)
print(asr_df.head(3))


Index(['audio_path', 'text', 'wav_path'], dtype='object')
                            audio_path  \
0  raw_audio/raw_audio/Speaker0_00.m4a   
1  raw_audio/raw_audio/Speaker0_01.m4a   
2  raw_audio/raw_audio/Speaker0_02.m4a   

                                                text                wav_path  
0                          Bugün hava oldukça sakin.  wav16k/Speaker0_00.wav  
1              Toplantı saatini üç buçuğa erteledim.  wav16k/Speaker0_01.wav  
2  Yoğun trafik nedeniyle otobüs yarım saatten fa...  wav16k/Speaker0_02.wav  


In [16]:
# Karakter setini çıkarıyoruz
texts = asr_df["text"].astype(str).tolist()

alphabet = sorted(set("".join(texts)))
print("Alfabe:", alphabet)
print("Karakter sayısı:", len(alphabet))

# CTC için 0 = blank
char2id = {c: i+1 for i, c in enumerate(alphabet)}
id2char = {i: c for c, i in char2id.items()}

VOCAB_SIZE = len(char2id) + 1
print("Vocab size:", VOCAB_SIZE)


Alfabe: [' ', ',', '-', '.', '?', 'A', 'B', 'D', 'E', 'G', 'K', 'M', 'O', 'P', 'S', 'T', 'V', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'y', 'z', '\xa0', 'â', 'ç', 'ö', 'ü', 'ğ', 'ı', 'ş', '’']
Karakter sayısı: 50
Vocab size: 51


In [17]:
def text_to_ids(text):
    return [char2id[c] for c in text]

asr_df["label"] = asr_df["text"].apply(text_to_ids)

# kontrol
asr_df[["text", "label"]].head(3)


,text,label
0,Bugün hava oldukça sakin.,"[7, 38, 25, 46, 32, 1, 26, 19, 39, 19, 1, 33, ..."
1,Toplantı saatini üç buçuğa erteledim.,"[16, 33, 34, 30, 19, 32, 37, 48, 1, 36, 19, 19..."
2,Yoğun trafik nedeniyle otobüs yarım saatten fa...,"[18, 33, 47, 38, 32, 1, 37, 35, 19, 24, 27, 29..."


In [18]:
import librosa
import numpy as np

SR = 16000
N_MELS = 80
N_FFT = 400
HOP = 160

def wav_to_logmel(path):
    y, sr = librosa.load(path, sr=SR)
    mel = librosa.feature.melspectrogram(
        y=y,
        sr=sr,
        n_fft=N_FFT,
        hop_length=HOP,
        n_mels=N_MELS
    )
    logmel = librosa.power_to_db(mel, ref=np.max)
    return logmel.astype(np.float32)

# sadece ilk wav üzerinde deneyelim
sample_path = asr_df["wav_path"].iloc[0]
spec = wav_to_logmel(sample_path)

print("Log-mel shape:", spec.shape)


Log-mel shape: (80, 408)


In [19]:
import numpy as np

MAX_T = 500

def pad_or_trim(spec, max_t=MAX_T):
    n_mels, t = spec.shape
    if t >= max_t:
        return spec[:, :max_t]
    pad = np.zeros((n_mels, max_t - t), dtype=spec.dtype)
    return np.concatenate([spec, pad], axis=1)

spec_fixed = pad_or_trim(spec)      # (80, 500)
cnn_input = spec_fixed[..., None]   # (80, 500, 1)

print("spec_fixed:", spec_fixed.shape)
print("cnn_input:", cnn_input.shape)


spec_fixed: (80, 500)
cnn_input: (80, 500, 1)


In [20]:
import numpy as np

MAX_T = 500

def pad_or_trim(spec, max_t=MAX_T):
    n_mels, t = spec.shape
    if t >= max_t:
        return spec[:, :max_t], max_t
    pad = np.zeros((n_mels, max_t - t), dtype=spec.dtype)
    return np.concatenate([spec, pad], axis=1), t

X = []
input_lengths = []

for p in asr_df["wav_path"]:
    spec = wav_to_logmel(p)
    spec_fixed, t = pad_or_trim(spec)
    X.append(spec_fixed)
    input_lengths.append(min(t, MAX_T))

X = np.array(X)[..., None]   # (N, 80, 500, 1)
input_lengths = np.array(input_lengths)

print("X shape:", X.shape)
print("input_lengths shape:", input_lengths.shape)


X shape: (100, 80, 500, 1)
input_lengths shape: (100,)


In [21]:
import tensorflow as tf
from tensorflow.keras import layers, models

inp = layers.Input(shape=(80, 500, 1), name="input")

x = layers.Conv2D(32, (3, 3), padding="same", activation="relu")(inp)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(64, (3, 3), padding="same", activation="relu")(x)
x = layers.MaxPooling2D((2, 2))(x)

# (batch, freq, time, ch) -> (batch, time, feat)
x = layers.Permute((2, 1, 3))(x)
x = layers.Reshape((500 // 4, -1))(x)

x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.3)(x)

y_pred = layers.Dense(VOCAB_SIZE, activation="softmax", name="y_pred")(x)

cnn_model = models.Model(inputs=inp, outputs=y_pred)
cnn_model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 80, 500, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 80, 500, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 40, 250, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 40, 250, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 20, 125, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ permute (Permute)               │ (None, 125, 20, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 125, 1280)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 125, 128)       │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 125, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ y_pred (Dense)                  │ (None, 125, 51)        │         6,579 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 189,363 (739.70 KB)

 Trainable params: 189,363 (739.70 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
import tensorflow as tf
from tensorflow.keras import layers, models

# CTC için ekstra inputlar
labels = layers.Input(name="labels", shape=(None,), dtype="int32")
input_len = layers.Input(name="input_length", shape=(1,), dtype="int32")
label_len = layers.Input(name="label_length", shape=(1,), dtype="int32")

ctc_loss = layers.Lambda(
    lambda x: tf.keras.backend.ctc_batch_cost(*x),
    name="ctc_loss"
)([labels, cnn_model.output, input_len, label_len])

ctc_model = models.Model(
    inputs=[cnn_model.input, labels, input_len, label_len],
    outputs=ctc_loss
)

ctc_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3))
ctc_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 80, 500,   │          0 │ -                 │
│                     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 80, 500,   │        320 │ input[0][0]       │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 40, 250,   │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 40, 250,   │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 20, 125,   │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ permute (Permute)   │ (None, 125, 20,   │          0 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 125, 1280) │          0 │ permute[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 125, 128)  │    163,968 │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 125, 128)  │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ labels (InputLayer) │ (None, None)      │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ y_pred (Dense)      │ (None, 125, 51)   │      6,579 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_length        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ label_length        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ctc_loss (Lambda)   │ (None, 1)         │          0 │ labels[0][0],     │
│                     │                   │            │ y_pred[0][0],     │
│                     │                   │            │ input_length[0][… │
│                     │                   │            │ label_length[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 189,363 (739.70 KB)

 Trainable params: 189,363 (739.70 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Train/Val split (X, labels_padded var)
X_tr, X_va, y_tr, y_va, ll_tr, ll_va = train_test_split(
    X, labels_padded, label_lengths,
    test_size=0.2, random_state=42
)

# input_length SABİT = 125
T_OUT = 125
in_tr = np.full((len(X_tr),), T_OUT, dtype=np.int32)
in_va = np.full((len(X_va),), T_OUT, dtype=np.int32)

# dense padded labels -> SparseTensor (0 paddingleri at)
def dense_to_sparse(dense, lengths):
    indices = []
    values = []
    for i, L in enumerate(lengths):
        for j in range(L):
            indices.append([i, j])
            values.append(dense[i, j])
    indices = np.array(indices, dtype=np.int64)
    values = np.array(values, dtype=np.int32)
    shape = np.array([dense.shape[0], int(lengths.max())], dtype=np.int64)
    return tf.SparseTensor(indices=indices, values=values, dense_shape=shape)

y_tr_sp = dense_to_sparse(y_tr, ll_tr)
y_va_sp = dense_to_sparse(y_va, ll_va)

print("Train size:", len(X_tr), "Val size:", len(X_va))


Train size: 80 Val size: 20


In [29]:
optimizer = tf.keras.optimizers.Adam(1e-3)

@tf.function
def train_step(x_batch, y_sparse, input_len, label_len):
    with tf.GradientTape() as tape:
        y_pred = cnn_model(x_batch, training=True)  # (B, 125, 51)
        # tf.nn.ctc_loss time-major ister: (T, B, C)
        logits = tf.transpose(y_pred, [1, 0, 2])
        loss = tf.nn.ctc_loss(
            labels=y_sparse,
            logits=logits,
            label_length=label_len,
            logit_length=input_len,
            logits_time_major=True,
            blank_index=0
        )
        loss = tf.reduce_mean(loss)

    grads = tape.gradient(loss, cnn_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, cnn_model.trainable_variables))
    return loss

@tf.function
def val_step(x_batch, y_sparse, input_len, label_len):
    y_pred = cnn_model(x_batch, training=False)
    logits = tf.transpose(y_pred, [1, 0, 2])
    loss = tf.nn.ctc_loss(
        labels=y_sparse,
        logits=logits,
        label_length=label_len,
        logit_length=input_len,
        logits_time_major=True,
        blank_index=0
    )
    return tf.reduce_mean(loss)

BATCH = 8
EPOCHS = 15

# dataset
train_ds = tf.data.Dataset.from_tensor_slices((X_tr,)).batch(BATCH)
val_ds   = tf.data.Dataset.from_tensor_slices((X_va,)).batch(BATCH)

for epoch in range(1, EPOCHS+1):
    # train
    tr_losses = []
    # batch batch sparse label üretmek için slice yapacağız
    for i, x_b in enumerate(train_ds):
        start = i*BATCH
        end = min((i+1)*BATCH, len(X_tr))

        y_sp = dense_to_sparse(y_tr[start:end], ll_tr[start:end])
        tr_loss = train_step(
            x_b, y_sp,
            tf.constant(in_tr[start:end]),
            tf.constant(ll_tr[start:end])
        )
        tr_losses.append(float(tr_loss))

    # val
    va_losses = []
    for i, x_b in enumerate(val_ds):
        start = i*BATCH
        end = min((i+1)*BATCH, len(X_va))

        y_sp = dense_to_sparse(y_va[start:end], ll_va[start:end])
        va_loss = val_step(
            x_b, y_sp,
            tf.constant(in_va[start:end]),
            tf.constant(ll_va[start:end])
        )
        va_losses.append(float(va_loss))

    print(f"Epoch {epoch}/{EPOCHS} - loss: {np.mean(tr_losses):.4f} - val_loss: {np.mean(va_losses):.4f}")


/usr/local/lib/python3.12/dist-packages/keras/src/models/functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input
Received: inputs=('Tensor(shape=(8, 80, 500, 1))',)
  warnings.warn(msg)
/usr/local/lib/python3.12/dist-packages/keras/src/models/functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input
Received: inputs=('Tensor(shape=(4, 80, 500, 1))',)
  warnings.warn(msg)


Epoch 1/15 - loss: 372.5606 - val_loss: 376.1632
Epoch 2/15 - loss: 371.2712 - val_loss: 376.1497
Epoch 3/15 - loss: 371.2159 - val_loss: 376.0368
Epoch 4/15 - loss: 370.4938 - val_loss: 374.0917
Epoch 5/15 - loss: 345.4471 - val_loss: 347.7647
Epoch 6/15 - loss: 341.4779 - val_loss: 347.7645
Epoch 7/15 - loss: 341.4777 - val_loss: 347.7645
Epoch 8/15 - loss: 341.4777 - val_loss: 347.7645
Epoch 9/15 - loss: 341.4777 - val_loss: 347.7645
Epoch 10/15 - loss: 341.4777 - val_loss: 347.7645
Epoch 11/15 - loss: 341.4777 - val_loss: 347.7645
Epoch 12/15 - loss: 341.4777 - val_loss: 347.7645
Epoch 13/15 - loss: 341.4777 - val_loss: 347.7645
Epoch 14/15 - loss: 341.4777 - val_loss: 347.7645
Epoch 15/15 - loss: 341.4777 - val_loss: 347.7645


In [31]:
# Text'leri de aynı random_state ile ayıralım
from sklearn.model_selection import train_test_split

_, text_va = train_test_split(
    asr_df["text"].values,
    test_size=0.2,
    random_state=42
)

print("text_va length:", len(text_va))
print(text_va[:3])


text_va length: 20
['Bu dosyayı ne zaman teslim etmemiz gerekiyor?'
 'Güneş enerjisi panelleri, sürdürülebilir enerji kaynakları arasında popülerliğini artırıyor.'
 'Yazılım mimarisi, ölçeklenebilir bir sistemin temelini oluşturur.']


In [32]:
pred_va = cnn_model.predict(X_va, verbose=0)
hyp = decode_batch(pred_va)

wers = [wer(r, h) for r, h in zip(text_va, hyp)]
print("VAL WER:", float(np.mean(wers)))

for i in range(min(5, len(hyp))):
    print("\nREF:", text_va[i])
    print("HYP:", hyp[i])


VAL WER: 1.0

REF: Bu dosyayı ne zaman teslim etmemiz gerekiyor?
HYP: 

REF: Güneş enerjisi panelleri, sürdürülebilir enerji kaynakları arasında popülerliğini artırıyor.
HYP: 

REF: Yazılım mimarisi, ölçeklenebilir bir sistemin temelini oluşturur.
HYP: 

REF: Dışarı çıkmadan önce havanın yağmur ihtimaline bakmayı unuttum.
HYP: 

REF: E-posta adresimi yanlış yazmış olabilirim, tekrar kontrol eder misin?
HYP: 


In [33]:
import re

def clean_text(t):
    t = t.lower()
    # noktalama ve özel işaretleri sil (Türkçe harfler kalsın)
    t = re.sub(r"[^\w\sçğıöşü]", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

asr_df["text_clean"] = asr_df["text"].astype(str).apply(clean_text)

print(asr_df[["text","text_clean"]].head(3))

alphabet2 = sorted(set("".join(asr_df["text_clean"].tolist())))
print("Eski karakter sayısı:", len(alphabet))
print("Yeni karakter sayısı:", len(alphabet2))


                                                text  \
0                          Bugün hava oldukça sakin.   
1              Toplantı saatini üç buçuğa erteledim.   
2  Yoğun trafik nedeniyle otobüs yarım saatten fa...   

                                          text_clean  
0                           bugün hava oldukça sakin  
1               toplantı saatini üç buçuğa erteledim  
2  yoğun trafik nedeniyle otobüs yarım saatten fa...  
Eski karakter sayısı: 50
Yeni karakter sayısı: 31


In [34]:
# 1) yeni alfabe ve sözlük
alphabet = sorted(set("".join(asr_df["text_clean"].tolist())))
char2id = {c: i+1 for i, c in enumerate(alphabet)}   # 0 blank
id2char = {i: c for c, i in char2id.items()}
VOCAB_SIZE = len(char2id) + 1

print("Yeni VOCAB_SIZE:", VOCAB_SIZE)
print("Örnek alfabe:", alphabet[:20])

# 2) yeni label'lar
def text_to_ids_clean(text):
    return [char2id[c] for c in text]

asr_df["label_clean"] = asr_df["text_clean"].apply(text_to_ids_clean)

# kontrol
asr_df[["text_clean", "label_clean"]].head(3)


Yeni VOCAB_SIZE: 32
Örnek alfabe: [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't']


,text_clean,label_clean
0,bugün hava oldukça sakin,"[3, 21, 8, 28, 15, 1, 9, 2, 22, 2, 1, 16, 13, ..."
1,toplantı saatini üç buçuğa erteledim,"[20, 16, 17, 13, 2, 15, 20, 30, 1, 19, 2, 2, 2..."
2,yoğun trafik nedeniyle otobüs yarım saatten fa...,"[23, 16, 29, 21, 15, 1, 20, 18, 2, 7, 10, 12, ..."


In [35]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

# 1) label padding ve uzunluklar
labels_padded = tf.keras.preprocessing.sequence.pad_sequences(
    asr_df["label_clean"], padding="post"
)
label_lengths = np.array([len(l) for l in asr_df["label_clean"]], dtype=np.int32)

print("labels_padded:", labels_padded.shape, "label_lengths:", label_lengths.shape)

# 2) CNN'in son katmanını yeni vocab'a göre değiştir
# (önceki cnn_model'dan gövdeyi alıyoruz)
feature_extractor = models.Model(
    inputs=cnn_model.input,
    outputs=cnn_model.layers[-3].output  # dropout'tan önceki Dense(128)
)

inp = feature_extractor.input
x = feature_extractor.output
y_pred = layers.Dense(VOCAB_SIZE, activation="softmax", name="y_pred_new")(x)

cnn_model_clean = models.Model(inputs=inp, outputs=y_pred)
cnn_model_clean.summary()

# 3) Train/Val split (text_clean ile uyumlu)
X_tr, X_va, y_tr, y_va, ll_tr, ll_va = train_test_split(
    X, labels_padded, label_lengths,
    test_size=0.2, random_state=42
)

# input_length sabit (CNN çıkışı 125)
T_OUT = 125
in_tr = np.full((len(X_tr),), T_OUT, dtype=np.int32)
in_va = np.full((len(X_va),), T_OUT, dtype=np.int32)


labels_padded: (100, 90) label_lengths: (100,)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 80, 500, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 80, 500, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 40, 250, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 40, 250, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 20, 125, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ permute (Permute)               │ (None, 125, 20, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 125, 1280)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 125, 128)       │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ y_pred_new (Dense)              │ (None, 125, 32)        │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 186,912 (730.12 KB)

 Trainable params: 186,912 (730.12 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
import tensorflow as tf
import numpy as np

optimizer = tf.keras.optimizers.Adam(3e-3)  # biraz daha agresif

# dense -> sparse yardımcı
def dense_to_sparse(dense, lengths):
    indices = []
    values = []
    for i, L in enumerate(lengths):
        for j in range(L):
            indices.append([i, j])
            values.append(dense[i, j])
    indices = np.array(indices, dtype=np.int64)
    values = np.array(values, dtype=np.int32)
    shape = np.array([dense.shape[0], int(lengths.max())], dtype=np.int64)
    return tf.SparseTensor(indices=indices, values=values, dense_shape=shape)

@tf.function
def train_step(x_batch, y_sparse, input_len, label_len):
    with tf.GradientTape() as tape:
        y_pred = cnn_model_clean(x_batch, training=True)  # (B,125,32)
        logits = tf.transpose(y_pred, [1, 0, 2])          # (125,B,32)
        loss = tf.nn.ctc_loss(
            labels=y_sparse,
            logits=logits,
            label_length=label_len,
            logit_length=input_len,
            logits_time_major=True,
            blank_index=0
        )
        loss = tf.reduce_mean(loss)
    grads = tape.gradient(loss, cnn_model_clean.trainable_variables)
    optimizer.apply_gradients(zip(grads, cnn_model_clean.trainable_variables))
    return loss

# dataset
BATCH = 8
EPOCHS = 10
T_OUT = 125

train_ds = tf.data.Dataset.from_tensor_slices((X_tr, y_tr, ll_tr)).batch(BATCH)
val_ds   = tf.data.Dataset.from_tensor_slices((X_va, y_va, ll_va)).batch(BATCH)

for epoch in range(1, EPOCHS+1):
    tr_losses = []
    for x_b, y_b, ll_b in train_ds:
        y_sp = dense_to_sparse(y_b.numpy(), ll_b.numpy())
        loss = train_step(
            x_b,
            y_sp,
            tf.fill([x_b.shape[0]], T_OUT),
            ll_b
        )
        tr_losses.append(float(loss))
    print(f"Epoch {epoch}/{EPOCHS} - loss: {np.mean(tr_losses):.3f}")


Epoch 1/10 - loss: 315.015
Epoch 2/10 - loss: 314.030
Epoch 3/10 - loss: 314.030
Epoch 4/10 - loss: 314.030
Epoch 5/10 - loss: 314.030
Epoch 6/10 - loss: 314.030
Epoch 7/10 - loss: 314.030
Epoch 8/10 - loss: 314.030
Epoch 9/10 - loss: 314.030
Epoch 10/10 - loss: 314.030


In [37]:
# decode
def decode_batch(pred):
    decoded, _ = tf.keras.backend.ctc_decode(
        pred,
        input_length=np.ones(pred.shape[0]) * pred.shape[1],
        greedy=True
    )
    decoded = decoded[0].numpy()
    outs = []
    for seq in decoded:
        chars = [id2char[i] for i in seq if i > 0 and i in id2char]
        outs.append("".join(chars))
    return outs

def wer(ref, hyp):
    r = ref.split()
    h = hyp.split()
    dp = np.zeros((len(r)+1, len(h)+1), dtype=int)
    for i in range(len(r)+1): dp[i,0] = i
    for j in range(len(h)+1): dp[0,j] = j
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            cost = 0 if r[i-1] == h[j-1] else 1
            dp[i,j] = min(dp[i-1,j]+1, dp[i,j-1]+1, dp[i-1,j-1]+cost)
    return dp[len(r), len(h)] / max(1, len(r))

# text_clean val set
_, text_clean_va = train_test_split(
    asr_df["text_clean"].values,
    test_size=0.2,
    random_state=42
)

pred_va = cnn_model_clean.predict(X_va, verbose=0)
hyp = decode_batch(pred_va)

wers = [wer(r, h) for r, h in zip(text_clean_va, hyp)]
print("VAL WER (clean):", float(np.mean(wers)))

for i in range(min(5, len(hyp))):
    print("\nREF:", text_clean_va[i])
    print("HYP:", hyp[i])


VAL WER (clean): 0.99

REF: bu dosyayı ne zaman teslim etmemiz gerekiyor
HYP: e

REF: güneş enerjisi panelleri sürdürülebilir enerji kaynakları arasında popülerliğini artırıyor
HYP: e

REF: yazılım mimarisi ölçeklenebilir bir sistemin temelini oluşturur
HYP: e

REF: dışarı çıkmadan önce havanın yağmur ihtimaline bakmayı unuttum
HYP: e

REF: e posta adresimi yanlış yazmış olabilirim tekrar kontrol eder misin
HYP: e
